In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
data_path_and_labels = pd.read_csv("../mapped_data/ant-1.7.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [84]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [85]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent'

In [86]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [87]:
d = pd.read_csv("../mapped_data/ant-1.7.csv")
d['data_path'][0]

'dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [88]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/ant-1.7/apache-ant-1.7.0/src/main/org/apache/tools/zip/ZipShort.java.restrictedcontent.embed'

In [89]:
d

,data_path,bugs
0,../dataset/promise/defect data/ant-1.7/apache-...,1
1,../dataset/promise/defect data/ant-1.7/apache-...,0
2,../dataset/promise/defect data/ant-1.7/apache-...,1
3,../dataset/promise/defect data/ant-1.7/apache-...,0
4,../dataset/promise/defect data/ant-1.7/apache-...,7
...,...,...
736,../dataset/promise/defect data/ant-1.7/apache-...,0
737,../dataset/promise/defect data/ant-1.7/apache-...,0
738,../dataset/promise/defect data/ant-1.7/apache-...,0
739,../dataset/promise/defect data/ant-1.7/apache-...,6


In [90]:
train, test = train_test_split(d, test_size=0.2)

In [91]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [72]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [92]:
train_x[0]

array([3111., 6996., 3111., ...,    0.,    0.,    0.])

In [73]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

10670.0

In [74]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [75]:
train_y

array([ 0,  0,  0,  3,  0,  0,  1,  2,  0,  0,  1,  1,  0,  0,  0,  0,  0,
        0,  1,  0,  0,  0,  4,  0,  0,  0,  1,  0,  4,  0,  0,  0,  0,  0,
        1,  0,  0,  6,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  8,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  5,  0,  1,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  1,  1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  2,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  2,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  2,
        0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        2,  0,  0,  0,  0,  0,  0,  0,  0,  3,  1,  0,  0,  0,  4,  0,  1,
        0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  3,  0,  0,  1,
        2,  1,  0,  1,  0,  0,  0,  1,  0,  0,  2,  3,  0,  0,  0,  0,  0,
        1,  0,  0,  0,  1

In [76]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [77]:
train_x.shape

(592, 3378)

In [78]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [79]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
19/19 [==============================] - 1s 24ms/step - loss: 1.4642 - mse: 1.4642 - mae: 0.6663 - root_mean_squared_error: 1.2101 - mean_squared_logarithmic_error: 0.2395
Epoch 2/100
19/19 [==============================] - 0s 19ms/step - loss: 1.2398 - mse: 1.2398 - mae: 0.6242 - root_mean_squared_error: 1.1135 - mean_squared_logarithmic_error: 0.1986
Epoch 3/100
19/19 [==============================] - 0s 19ms/step - loss: 1.0693 - mse: 1.0693 - mae: 0.5876 - root_mean_squared_error: 1.0341 - mean_squared_logarithmic_error: 0.1718
Epoch 4/100
19/19 [==============================] - 0s 20ms/step - loss: 0.9554 - mse: 0.9554 - mae: 0.5095 - root_mean_squared_error: 0.9775 - mean_squared_logarithmic_error: 0.1501
Epoch 5/100
19/19 [==============================] - 0s 20ms/step - loss: 0.9264 - mse: 0.9264 - mae: 0.4957 - root_mean_squared_error: 0.9625 - mean_squared_logarithmic_error: 0.1567
Epoch 6/100
19/19 [==============================] - 0s 19ms/step - loss: 0.8509

19/19 [==============================] - 0s 20ms/step - loss: 0.0939 - mse: 0.0939 - mae: 0.1368 - root_mean_squared_error: 0.3064 - mean_squared_logarithmic_error: 0.0270
Epoch 45/100
19/19 [==============================] - 0s 20ms/step - loss: 0.0894 - mse: 0.0894 - mae: 0.1282 - root_mean_squared_error: 0.2990 - mean_squared_logarithmic_error: 0.0255
Epoch 46/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0828 - mse: 0.0828 - mae: 0.1155 - root_mean_squared_error: 0.2877 - mean_squared_logarithmic_error: 0.0223
Epoch 47/100
19/19 [==============================] - 0s 20ms/step - loss: 0.0770 - mse: 0.0770 - mae: 0.1147 - root_mean_squared_error: 0.2775 - mean_squared_logarithmic_error: 0.0222
Epoch 48/100
19/19 [==============================] - 0s 22ms/step - loss: 0.0734 - mse: 0.0734 - mae: 0.1105 - root_mean_squared_error: 0.2710 - mean_squared_logarithmic_error: 0.0212
Epoch 49/100
19/19 [==============================] - 0s 25ms/step - loss: 0.0679 - mse:

19/19 [==============================] - 0s 23ms/step - loss: 0.0194 - mse: 0.0194 - mae: 0.0371 - root_mean_squared_error: 0.1395 - mean_squared_logarithmic_error: 0.0073
Epoch 89/100
19/19 [==============================] - 0s 21ms/step - loss: 0.0184 - mse: 0.0184 - mae: 0.0317 - root_mean_squared_error: 0.1357 - mean_squared_logarithmic_error: 0.0069
Epoch 90/100
19/19 [==============================] - 0s 22ms/step - loss: 0.0179 - mse: 0.0179 - mae: 0.0316 - root_mean_squared_error: 0.1339 - mean_squared_logarithmic_error: 0.0068
Epoch 91/100
19/19 [==============================] - 0s 20ms/step - loss: 0.0179 - mse: 0.0179 - mae: 0.0286 - root_mean_squared_error: 0.1338 - mean_squared_logarithmic_error: 0.0066
Epoch 92/100
19/19 [==============================] - 0s 19ms/step - loss: 0.0174 - mse: 0.0174 - mae: 0.0295 - root_mean_squared_error: 0.1319 - mean_squared_logarithmic_error: 0.0066
Epoch 93/100
19/19 [==============================] - 0s 20ms/step - loss: 0.0173 - mse:

In [80]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:06:09 ago. (Use '!kill 21467' to kill it.)

In [81]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 0s 6ms/step - loss: 0.4986 - mse: 0.4986 - mae: 0.4280 - root_mean_squared_error: 0.7061 - mean_squared_logarithmic_error: 0.1448


[0.49856361746788025,
 0.49856361746788025,
 0.42801138758659363,
 0.7060903906822205,
 0.1448075771331787]

In [23]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
0,531.0,506.0,1966.0,1734.0,1956.0,1734.0,2724.0,1734.0,1592.0,1865.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,533.0,847.0,1058.0,847.0,1606.0,1734.0,1174.0,1734.0,1173.0,870.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2102.0,1865.0,1617.0,870.0,2037.0,1734.0,1824.0,1734.0,1865.0,2259.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,530.0,1734.0,136.0,1734.0,134.0,136.0,279.0,1701.0,416.0,2063.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,534.0,1734.0,328.0,1734.0,330.0,1734.0,329.0,847.0,1058.0,1734.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,468.0,2414.0,2119.0,504.0,1492.0,504.0,1931.0,847.0,802.0,504.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,2225.0,957.0,2119.0,2414.0,2119.0,504.0,2703.0,2119.0,959.0,2119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,469.0,469.0,468.0,504.0,2581.0,1508.0,1678.0,2581.0,844.0,2262.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,361.0,504.0,1622.0,504.0,1314.0,578.0,2661.0,886.0,264.0,362.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
0,0,531.0,506.0,1966.0,1734.0,1956.0,1734.0,2724.0,1734.0,1592.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,533.0,847.0,1058.0,847.0,1606.0,1734.0,1174.0,1734.0,1173.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2102.0,1865.0,1617.0,870.0,2037.0,1734.0,1824.0,1734.0,1865.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,530.0,1734.0,136.0,1734.0,134.0,136.0,279.0,1701.0,416.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,534.0,1734.0,328.0,1734.0,330.0,1734.0,329.0,847.0,1058.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0,468.0,2414.0,2119.0,504.0,1492.0,504.0,1931.0,847.0,802.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,0,2225.0,957.0,2119.0,2414.0,2119.0,504.0,2703.0,2119.0,959.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,0,469.0,469.0,468.0,504.0,2581.0,1508.0,1678.0,2581.0,844.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0,361.0,504.0,1622.0,504.0,1314.0,578.0,2661.0,886.0,264.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
train, test = train_test_split(data, test_size=0.2)

In [26]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
43,1,225.0,1103.0,1210.0,1678.0,1103.0,844.0,2647.0,75.0,1851.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118,0,311.0,2225.0,504.0,2119.0,2414.0,2119.0,504.0,2703.0,2119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,1,116.0,115.0,2752.0,115.0,115.0,1785.0,115.0,512.0,2577.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,0,201.0,504.0,2055.0,225.0,2058.0,225.0,1103.0,847.0,1242.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0,33.0,504.0,2558.0,1721.0,33.0,1028.0,2410.0,2595.0,2447.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,3,569.0,504.0,1148.0,538.0,2211.0,578.0,910.0,578.0,569.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,530.0,1734.0,136.0,1734.0,134.0,136.0,279.0,1701.0,416.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0,539.0,412.0,2181.0,1210.0,373.0,1213.0,2703.0,101.0,874.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,0,504.0,1942.0,2389.0,2741.0,504.0,1942.0,2741.0,1210.0,1678.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604
47,0,225.0,2507.0,225.0,1094.0,847.0,1275.0,847.0,1303.0,2437.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0,338.0,504.0,1258.0,1734.0,1830.0,1734.0,947.0,338.0,565.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,0,115.0,938.0,115.0,504.0,1050.0,504.0,2080.0,504.0,952.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,1,225.0,831.0,504.0,922.0,225.0,2490.0,504.0,2553.0,435.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0,320.0,1210.0,847.0,1854.0,357.0,115.0,938.0,2473.0,2703.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0,317.0,116.0,941.0,116.0,847.0,1304.0,225.0,1103.0,225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,0,225.0,1078.0,847.0,1083.0,799.0,1153.0,225.0,2471.0,225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0,462.0,554.0,554.0,416.0,2058.0,1181.0,462.0,2058.0,416.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,0,504.0,580.0,504.0,207.0,504.0,528.0,504.0,504.0,408.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,0,504.0,1776.0,504.0,786.0,504.0,1807.0,504.0,2535.0,504.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [33]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [29]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [30]:
X_train = train.values
X_test = test.values

In [31]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [37]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [38]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
4/4 [==============================] - 1s 3ms/step - loss: 0.6660 - mse: 0.6660 - mae: 0.3932 - root_mean_squared_error: 0.8161 - mean_squared_logarithmic_error: 0.1991
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 0.5165 - mse: 0.5165 - mae: 0.2805 - root_mean_squared_error: 0.7187 - mean_squared_logarithmic_error: 0.1365
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 0.5161 - mse: 0.5161 - mae: 0.2733 - root_mean_squared_error: 0.7184 - mean_squared_logarithmic_error: 0.1428
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 0.5935 - mse: 0.5935 - mae: 0.3295 - root_mean_squared_error: 0.7704 - mean_squared_logarithmic_error: 0.1699
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 0.4284 - mse: 0.4284 - mae: 0.2649 - root_mean_squared_error: 0.6545 - mean_squared_logarithmic_error: 0.1288
Epoch 6/100
4/4 [==============================] - 0s 3ms/step - loss: 0.4706 - mse: 0.4706 - m

4/4 [==============================] - 0s 3ms/step - loss: 0.4131 - mse: 0.4131 - mae: 0.3148 - root_mean_squared_error: 0.6427 - mean_squared_logarithmic_error: 0.1234
Epoch 91/100
4/4 [==============================] - 0s 3ms/step - loss: 0.6215 - mse: 0.6215 - mae: 0.4890 - root_mean_squared_error: 0.7883 - mean_squared_logarithmic_error: 0.2116
Epoch 92/100
4/4 [==============================] - 0s 3ms/step - loss: 0.4589 - mse: 0.4589 - mae: 0.3426 - root_mean_squared_error: 0.6774 - mean_squared_logarithmic_error: 0.1409
Epoch 93/100
4/4 [==============================] - 0s 3ms/step - loss: 0.4499 - mse: 0.4499 - mae: 0.2428 - root_mean_squared_error: 0.6707 - mean_squared_logarithmic_error: 0.1124
Epoch 94/100
4/4 [==============================] - 0s 3ms/step - loss: 0.5151 - mse: 0.5151 - mae: 0.2584 - root_mean_squared_error: 0.7177 - mean_squared_logarithmic_error: 0.1302
Epoch 95/100
4/4 [==============================] - 0s 3ms/step - loss: 0.4610 - mse: 0.4610 - mae: 0.2

In [39]:
score = model.evaluate(test_x, test_y)
score

1/1 [==============================] - 0s 377ms/step - loss: 0.2706 - mse: 0.2706 - mae: 0.2159 - root_mean_squared_error: 0.5202 - mean_squared_logarithmic_error: 0.0988


[0.2706144154071808,
 0.2706144154071808,
 0.21585232019424438,
 0.5202061533927917,
 0.09882837533950806]